# CPRD Notebook:
## Evaluation of the pre-trained SurvivEHR-CR model on a supervised cohort study, with no additional training (zero-shot).

Cohort study: predicting Cardiovascular Disease in a Type 2 Diabetes Mellitus population.

This notebook quantifies the performance obtained when applying the pre-trained model to a sub-population, without fine-tuning

In [14]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import logging
import wandb
from tqdm import tqdm
import pickle
from hydra import compose, initialize
from omegaconf import OmegaConf
from CPRD.examples.modelling.SurvStreamGPT.run_experiment import run
from CPRD.data.foundational_loader import FoundationalDataModule
from CPRD.src.models.survival.task_heads.causal import SurvStreamGPTForCausalModelling

import time
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import os
import polars as pl
pl.Config.set_tbl_rows(10000)
import pandas as pd
pd.options.display.max_rows = 10000

torch.manual_seed(1337)
torch.set_float32_matmul_precision('medium')

logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")

 # TODO: define an env variable to fix for a local hpc environment issue, this shouldn't be needed
%env SLURM_NTASKS_PER_NODE=28   

Using device: cuda.
env: SLURM_NTASKS_PER_NODE=28


## Choosing configurations
The default configuration is for pre-training. Here we modify as necesssary

Here we choose to load in the configuration for a small **pre-trained** 11.4M parameter model, named "CR_11M". We specfiy the `zero-shot` experiment type, which will lead to running the ```SupervisedExperiment```. 

We tell this experiment that no further training is needed. Additionally, we do choose to perform testing (true by default). As this is a supervised model, this tests the ability to predict the outcomes of interest. In this notebook, this is chosen to be those of the cohort study for predicting Cardiovascular Disease in a Type 2 Diabetes Mellitus population, and we add the folder containing this dataset to the configuration. 

```Note: As this is a supervised dataset, we need to tell the DataModule that the last event observed is a target and must be stripped. This is done by passing a list of targets to the configuration, overriding the null default. This lets the DataModule know that it should process batches as supervised.```

We set the number of workers to be appropriate for the number of CPUs available to reduce bottlenecking, and tell the experiment that we do not want to limit the number of testing batches. In addition, we specify where we want any checkpoints to be saved to avoid bloating the repository.

# Run small (11M) model experiment

```
Small 11M model

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Test:OutcomePerformanceMetricsctd  │         0.6068708896636963          │
│  Test:OutcomePerformanceMetricsibs  │        0.035581761215435055         │
│ Test:OutcomePerformanceMetricsinbll │         0.25309925866451416         │
│              test_loss              │          8.996182441711426          │
│          test_loss_desurv           │         18.201364517211914          │
│          test_loss_values           │        -0.20899981260299683         │
└─────────────────────────────────────┴─────────────────────────────────────┘

annealed scheduler
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Test:OutcomePerformanceMetricsctd  │          0.614702582359314          │
│  Test:OutcomePerformanceMetricsibs  │         0.03559818346555152         │
│ Test:OutcomePerformanceMetricsinbll │         0.26287718977162844         │
│              test_loss              │          9.010758399963379          │
│          test_loss_desurv           │         18.226964950561523          │
│          test_loss_values           │        -0.20544125139713287         │
└─────────────────────────────────────┴─────────────────────────────────────┘


```

In [15]:
# load the configuration file, override any settings 
with initialize(version_base=None, config_path="../../confs", job_name="testing_notebook"):
    cfg = compose(config_name="config_CompetingRisk11M", 
                  overrides=[# Experiment setup
                             "experiment.type='zeroshot'",
                             "experiment.run_id='CR_11M'",
                             "experiment.fine_tune_id='CVD_0shot'",
                             "experiment.train=False",
                             "experiment.test=True",
                             'experiment.fine_tune_outcomes=["IHDINCLUDINGMI_OPTIMALV2", "ISCHAEMICSTROKE_V2", "MINFARCTION", "STROKEUNSPECIFIED_V2", "STROKE_HAEMRGIC"]',
                             # Dataloader
                             "data.path_to_ds=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/",
                             "data.meta_information_path=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle",
                             "data.min_workers=12",
                             # Optimiser
                             "optim.limit_test_batches=null",
                            ]
                 )     

# print(OmegaConf.to_yaml(cfg))
model, dm = run(cfg)
print(f"Loaded model with {sum(p.numel() for p in model.parameters())/1e6} M parameters")

wandb.finish()


INFO:root:Running cr on 72 CPUs and 1 GPUs
INFO:root:====================================================================================================
INFO:root:# Loading DataModule for dataset /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/. This will be loaded in supervised form.
INFO:root:====================================================================================================
INFO:root:Creating supervised collator for DataModule
INFO:root:Using meta information from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle
INFO:root:Using train file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_train.pickle
INFO:root:Using test file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_t

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Test:OutcomePerformanceMetricsctd  │         0.6127204895019531          │
│  Test:OutcomePerformanceMetricsibs  │         0.03558166821578663         │
│ Test:OutcomePerformanceMetricsinbll │         0.2495998501393419          │
│              test_loss              │          9.032392501831055          │
│          test_loss_desurv           │         18.286590576171875          │
│          test_loss_values           │         -0.2218271940946579         │
└─────────────────────────────────────┴─────────────────────────────────────┘

Loaded model with 11.433294 M parameters


Test:OutcomePerformanceMetricsctd,▁
Test:OutcomePerformanceMetricsibs,▁
Test:OutcomePerformanceMetricsinbll,▁
epoch,▁
test_loss,▁
test_loss_desurv,▁
test_loss_values,▁
trainer/global_step,▁
Test:OutcomePerformanceMetricsctd,0.61272
Test:OutcomePerformanceMetricsibs,0.03558
Test:OutcomePerformanceMetricsinbll,0.2496


# Repeat this for the 37M parameter model

```
partial completion of new 129M model
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Test:OutcomePerformanceMetricsctd  │          0.601206362247467          │
│  Test:OutcomePerformanceMetricsibs  │        0.035615343504889076         │
│ Test:OutcomePerformanceMetricsinbll │         0.2714460757858094          │
│              test_loss              │          8.885648727416992          │
│          test_loss_desurv           │          17.97450828552246          │
│          test_loss_values           │         -0.2032042294740677         │
└─────────────────────────────────────┴─────────────────────────────────────┘
```

In [3]:
# load the configuration file, override any settings 
with initialize(version_base=None, config_path="../../confs", job_name="testing_notebook"):
    cfg = compose(config_name="config_CompetingRisk37M", 
                  overrides=[# Experiment setup
                             "experiment.type='zeroshot'",
                             "experiment.run_id='CR_37M'",
                             "experiment.fine_tune_id='CVD_0shot'",
                             "experiment.train=False",
                             "experiment.test=True",
                             'experiment.fine_tune_outcomes=["IHDINCLUDINGMI_OPTIMALV2", "ISCHAEMICSTROKE_V2", "MINFARCTION", "STROKEUNSPECIFIED_V2", "STROKE_HAEMRGIC"]',
                             # Dataloader
                             "data.path_to_ds=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/",
                             "data.meta_information_path=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle",
                             "data.min_workers=12",
                             # Optimiser
                             "optim.limit_test_batches=null",
                            ]
                 )     

model, dm = run(cfg)
print(f"Loaded model with {sum(p.numel() for p in model.parameters())/1e6} M parameters")

wandb.finish()

INFO:root:Running cr on 72 CPUs and 1 GPUs
INFO:root:====================================================================================================
INFO:root:# Loading DataModule for dataset /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/. This will be loaded in supervised form.
INFO:root:====================================================================================================
INFO:root:Creating supervised collator for DataModule
INFO:root:Using meta information from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle
INFO:root:Using train file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_train.pickle
INFO:root:Using test file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_t

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Test:OutcomePerformanceMetricsctd  │          0.58172208070755           │
│  Test:OutcomePerformanceMetricsibs  │        0.035216640838986284         │
│ Test:OutcomePerformanceMetricsinbll │         0.19741412818471304         │
│              test_loss              │         15.430459976196289          │
│          test_loss_desurv           │          17.1679630279541           │
│          test_loss_values           │        -0.20718777179718018         │
└─────────────────────────────────────┴─────────────────────────────────────┘

Loaded model with 18.455118 M parameters


Test:OutcomePerformanceMetricsctd,▁
Test:OutcomePerformanceMetricsibs,▁
Test:OutcomePerformanceMetricsinbll,▁
epoch,▁
test_loss,▁
test_loss_desurv,▁
test_loss_values,▁
trainer/global_step,▁
Test:OutcomePerformanceMetricsctd,0.58172
Test:OutcomePerformanceMetricsibs,0.03522
Test:OutcomePerformanceMetricsinbll,0.19741


In [8]:
# import wandb
print(model)
# wandb.finish()

SurvStreamGPTForCausalModelling(
  (transformer): TTETransformer(
    (wpe): TemporalPositionalEncoding()
    (wte): DataEmbeddingLayer(
      (static_proj): Linear(in_features=16, out_features=1024, bias=True)
      (dynamic_embedding_layer): SplitDynamicEmbeddingLayer(
        (cat_event_embed_layer): Embedding(265, 1024, padding_idx=0)
        (cat_event_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (num_value_embed_layer): EmbeddingBag(265, 1024, mode='sum', padding_idx=0)
        (num_value_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
    )
    (drop): Dropout(p=0.2, inplace=False)
    (blocks): ModuleList(
      (0-9): 10 x Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadedSelfAttention(
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
    

In [ ]:
dm.encode(['IHDINCLUDINGMI_OPTIMALV2', 'ISCHAEMICSTROKE_V2', 'MINFARCTION', 'STROKEUNSPECIFIED_V2', 'STROKE_HAEMRGIC'])
# display(dm.decode([95, 175, 263,249]).split(" "))

In [ ]:
dm.tokenizer._event_counts["EVENT"][-5:].to_list()

In [15]:
save_path = f"/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/{cfg.experiment.run_id}/"

# Load Pre-Trained model

In [ ]:
ckpt_path = cfg.experiment.log_dir + f'checkpoints/{cfg.experiment.run_id}.ckpt'
model = SurvivalExperiment.load_from_checkpoint(ckpt_path)

# Initialise fine-tuning data module

In [5]:
# Update dataset path to point to the new dataset 
cfg.data.path_to_ds = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/"

# Build 
dm = FoundationalDataModule(path_to_db=cfg.data.path_to_db,
                            path_to_ds=cfg.data.path_to_ds,
                            load=True,
                            tokenizer="tabular",
                            batch_size=cfg.data.batch_size,
                            max_seq_length=cfg.transformer.block_size,
                            freq_threshold=cfg.data.unk_freq_threshold,
                            min_workers=cfg.data.min_workers,
                            overwrite_meta_information=cfg.data.meta_information_path,
                           )

vocab_size = dm.train_set.tokenizer.vocab_size
print(f"{vocab_size} vocab elements")

# list of univariate measurements to model with Normal distribution
# Extract the measurements, using the fact that the diagnoses are all up upper case.
measurements_for_univariate_regression = [record for record in dm.tokenizer._event_counts["EVENT"] if record.upper() != record]
cfg.head.tokens_for_univariate_regression = dm.encode(measurements_for_univariate_regression) 
# display(measurements_for_univariate_regression)

INFO:root:Creating unsupervised collator for DataModule
INFO:root:Using meta information from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle
INFO:root:Using train file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_train.pickle
INFO:root:Using test file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_test.pickle
INFO:root:Using val file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_val.pickle
INFO:root:Tokenzier created based on 7,555,415,275 tokens
INFO:root:Using tabular tokenizer, created from meta information and containing 265 tokens
INFO:root:Loaded /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_C

265 vocab elements


In [37]:
num_diagnoses = len(dm.train_set.meta_information["diagnosis_table"]["count"])
num_diagnosis_events = sum(dm.train_set.meta_information["diagnosis_table"]["count"])

is_medication = dm.train_set.meta_information["measurement_tables"]["count_obs"] == 0

num_medications = sum(is_medication)
num_medication_events = sum(dm.train_set.meta_information["measurement_tables"][is_medication]["count"])
num_measurement_test = sum(~is_medication)
num_measurement_test_events = sum(dm.train_set.meta_information["measurement_tables"][~is_medication]["count"])

num_measurement_test_events = sum(dm.train_set.meta_information["measurement_tables"][~is_medication]["count_obs"])

print(f'{num_diagnosis_events:,} diagnoses of {num_diagnoses} types')
print(f'{num_medication_events:,} medications of {num_medications} types')
print(f'{num_measurement_test_events:,} measurements and tests of {num_measurement_test} types')
print(f'{num_diagnoses+num_medication_events+num_measurement_test_events:,}')

print(f'{num_measurement_test_events:,}')
dm.train_set.meta_information.keys()

print(dm.train_set.tokenizer._event_counts)

51,003,640 diagnoses of 74 types
3,970,984,804 medications of 81 types
3,364,437,065 measurements and tests of 108 types
7,335,421,943
3,364,437,065
shape: (264, 3)
┌───────────────────────────────────┬───────────┬───────────┐
│ EVENT                             ┆ COUNT     ┆ FREQUENCY │
│ ---                               ┆ ---       ┆ ---       │
│ str                               ┆ u32       ┆ f64       │
╞═══════════════════════════════════╪═══════════╪═══════════╡
│ UNK                               ┆ 0         ┆ 0.0       │
│ ADDISONS_DISEASE                  ┆ 6691      ┆ 8.8559e-7 │
│ CYSTICFIBROSIS                    ┆ 7053      ┆ 9.3350e-7 │
│ SYSTEMIC_SCLEROSIS                ┆ 8772      ┆ 0.000001  │
│ SICKLE_CELL_DISEASE_V2            ┆ 11159     ┆ 0.000001  │
│ ADDISON_DISEASE                   ┆ 11794     ┆ 0.000002  │
│ DOWNSSYNDROME                     ┆ 17006     ┆ 0.000002  │
│ HAEMOCHROMATOSIS_V2               ┆ 18631     ┆ 0.000002  │
│ PLASMACELL_NEOPLASM_V2     

In [ ]:
# # import pickle as pkl
# # import pathlib

# pkl_file_to_amend = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_test.pickle"

# with open(pkl_file_to_amend, 'rb') as pickle_file:
#     content = pickle.load(pickle_file)
# display(content)

# # new_dictionary = {}
# # for key in content.keys():
# #     str_to_remove = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/split=val/"
# #     new_key = str(key)[len(str_to_remove):]
# #     new_dictionary[new_key] = content[key]
# # display(new_dictionary)


# # with open(pkl_file_to_amend, 'wb') as handle:
# #     pickle.dump(new_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# new_dictionary

In [ ]:
import copy
start = time.time()   # starting time
for batch in dm.train_dataloader():
    # print(batch["tokens"][1,:])
    
    c_batch = convert_batch_to_none_causal(batch)
    # print(c_batch["tokens"][1,:])
    # print(c_batch["target_token"][1])

    # print(batch["tokens"][1,:])
    
    break
    
print(f"batch loaded in {time.time()-start} seconds")    
    
# for key in batch.keys():
#     print(f"{key}".ljust(20) + f"{batch[key].shape}")

# tokens = batch["tokens"][0].tolist()    
# sentence = dm.decode(tokens).split(" ")
# for token, value in zip(sentence, batch["values"][0].tolist()):
#     print(f"{token}:".ljust(40) + f"{value}")

In [ ]:
display(batch.keys())
display(c_batch.keys())

print(batch["static_covariates"].shape)

# print(dm.train_set.static_1hot)
# print(dm.train_set.static_1hot["SEX"].categories_)
# print(dm.train_set.static_1hot["IMD"].categories_)
# print(dm.train_set.static_1hot["ETHNICITY"].categories_)

print(batch["tokens"][1,:])
print(c_batch["tokens"][1,:])
print(c_batch["target_token"][1])

## View an example sample

In [ ]:
dm.test_set.view_sample(11003, max_dynamic_events=None, report_time=True)

# Custom wrapper prediction last token

To begin with, I will just loop over samples individually to test the zero-shot capacity of SurvivEHR. 

In [ ]:


# Verifying on datamodule 
for _idx, batch in enumerate(dm.test_dataloader()):
    if _idx > 10:
        break
    print(_idx)
    print(torch.stack([batch["tokens"][10,:5], 
                       batch["values"][10,:5],  
                       batch["ages"][10,:5],
                       batch["attention_mask"][10,:5]]))
    batch = replace_last_non_pad_with_pad(batch)
    print(torch.stack([batch["tokens"][10,:5], 
                       batch["values"][10,:5],  
                       batch["ages"][10,:5],
                       batch["attention_mask"][10,:5]]))

In [ ]:
outcome_of_interest = ["COPD", "SUBSTANCEMISUSE"]
outcome_token = dm.encode(outcome_of_interest)[0]
print(outcome_token)
# print(model(batch))

In [ ]:
Hs, labels = [], []
mins,maxes=[],[]
for _idx, batch in enumerate(dm.test_dataloader()):

    batch = replace_last_non_pad_with_pad(batch)
    print(batch["tokens".shape)
    outputs, _, hidden_states = model(batch, is_generation=True)
    print(outputs)
    
    hidden_states = hidden_states.cpu().detach().numpy()                           # (64, 128, 384) 
    Hs.append( hidden_states.reshape(hidden_states.shape[0], -1) )
    labels.append((batch["target_token"] == outcome_token).long().numpy())

    if _idx == 9:
        break



# Visualise hidden dimension labelled by target

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler

H = np.concatenate(Hs, 0)
lbl = np.concatenate(labels, 0)

H = StandardScaler().fit_transform(H)
reducer = umap.UMAP()
H_proj = reducer.fit_transform(H)

plt.close()
plt.scatter(H_proj[:,0], H_proj[:,1], c=lbl)
plt.savefig(save_path + f"zero_shot/hidden_umap.png")

In [ ]:
print(outputs["surv"]["surv_CDF"][outcome_token].shape)

# The first two tokens in the vocab correspond to the PAD and UNK tokens. There is no CDF corresponding to the PAD token, so the indexing for surv_CDF begins as ["UNK", "ADDISONS_DISEASE", ...]
# print(dm.decode([0,1,2]))

outcomes = ["COPD", "SUBSTANCEMISUSE"]
outcome_tokens = dm.encode(outcomes)

# for outcome in outcomes:
    # observed_outcome_token = dm.encode([outcome])[0]
cdf = np.zeros_like(outputs["surv"]["surv_CDF"][0])
lbls = np.zeros_like(batch["target_token"])

for _outcome_token in outcome_tokens:
    cdf += outputs["surv"]["surv_CDF"][_outcome_token - 1] 
    lbls += (batch["target_token"] == _outcome_token).long().numpy()

plt.close()
cdf_true = cdf[lbls==1,:]
cdf_false = cdf[lbls==0,:]
for i in range(cdf_true.shape[0]):
    plt.plot(np.linspace(1,1826,1826), cdf_true[i,:], c="r", label="outcome occurred next" if i == 0 else None, alpha=1)
for i in range(cdf_false.shape[0]):
    plt.plot(np.linspace(1,1826,1826), cdf_false[i,:], c="k", label="outcome did not occur next" if i == 0 else None, alpha=0.3)

plt.legend(loc=2)
plt.xlabel("days")
plt.ylabel(f"P(t>T) - outcomes={','.join(outcomes)}")
plt.savefig(save_path + f"zero_shot/cdf_outcomes.png")

In [ ]:
print(batch["target_token"].unique())
print(len(outputs["surv"]["surv_CDF"]))

In [ ]:
dm.decode([2])

In [ ]:
outputs["surv"]["surv_CDF"][observed_outcome_token - 1]